In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bioage_framework.chat_model import ChatModel
import config

### Initialize a chatbot and test that it works

In [3]:
chatbot = ChatModel(url=config.URL_CB, api_key=config.API_KEY_CB, chatbot_id=config.ID_CB)
res = chatbot.query('What does an increased level of albumin mean?')
print(res)

(True, 'An increased level of albumin in the blood typically indicates good nutritional status and adequate intake of dietary proteins. It suggests that the body is able to maintain fluid balance and proper functioning of tissues.')


### Get a bioage model

In [5]:
from pathlib import Path
import os
import shap
import numpy as np
import pickle
import torch
from txai_omics_3.src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel

root_dir = Path(os.getcwd()).parent

# Model
fn_model = f"{root_dir}/samples/txai_omics_3/data/immuno/model.ckpt"
model = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=fn_model)


# SHAP
fn_shap = f"{root_dir}/samples/txai_omics_3/data/immuno/shap.pickle"

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
with open(fn_shap, 'rb') as handle:
    shap_dict = pickle.load(handle)

Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.2.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint h:\Lobachevsky\FrameworkLib\samples\txai_omics_3\data\immuno\model.ckpt`


### Load data

In [6]:
import pandas as pd

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

my_df = pd.DataFrame(my_data, index=[0])

### Initialize a connector

In [7]:
from bioage_framework.connector import Connector

connector = Connector(bioage_model=model, chat_model=chatbot)

### Run an analysis task

In [8]:
res = connector.analyze(data=my_df, shap_dict=shap_dict)
print(res)

You biological age is 79, and you aging acceleration is -11, which means you are ageing slower than normal.

Here is some more information about your data. 

CXCL9: 11.461565454538414
An increased level of CXCL9 indicates an activation of the inflammatory immune response and is associated with systemic chronic inflammation, aging of the cardiovascular system, and impaired vascular elasticity.

CD40LG: 9.55185262067581
Hmm, I am not sure.


